In [4]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, col, count, sum, to_date


spark = SparkSession.builder.appName('records').getOrCreate()

data_df = spark.read.csv('web_server_logs.csv', header=True, inferSchema=True)

data_df = data_df.withColumn('timestamp', to_date('timestamp', 'yyyy-mm-dd' ))

top_ip = data_df.groupBy('ip').count().withColumnRenamed('count', 'request_count').orderBy(col('request_count').desc()).limit(10)
top_ip.show()

methods = data_df.groupBy('method').count().withColumnRenamed("count","method_count").orderBy(col('method_count').desc())
methods.show()

num_of_404 = data_df.filter(col('response_code')==404).groupBy('timestamp').agg(sum(col('response_size')).alias('response_size')).orderBy('timestamp')
num_of_404.show()

+---------------+-------------+
|             ip|request_count|
+---------------+-------------+
|   88.104.65.46|            2|
| 214.177.57.171|            2|
| 114.16.181.188|            1|
|  50.247.213.23|            1|
| 51.226.248.128|            1|
| 112.135.37.215|            1|
|183.193.162.201|            1|
|    56.45.59.90|            1|
|  46.180.172.64|            1|
|  90.118.185.90|            1|
+---------------+-------------+

+------+------------+
|method|method_count|
+------+------------+
|   GET|       25062|
|   PUT|       25023|
|DELETE|       24969|
|  POST|       24946|
+------+------------+

+----------+-------------+
| timestamp|response_size|
+----------+-------------+
|2024-01-01|       372550|
|2024-01-02|       487168|
|2024-01-03|       405960|
|2024-01-04|       474136|
|2024-01-05|       394010|
|2024-01-06|       425517|
|2024-01-07|       468646|
|2024-01-08|       456236|
|2024-01-09|       508468|
|2024-01-10|       348437|
|2024-01-11|       4823